In [21]:

import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

source_folders = ["images/training/deforestation", "images/training/not_deforestation"]

# Load a pretrained ResNet model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode (important for inference)

# Preprocess the image (resize, normalize, convert to tensor)
transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image
    transforms.CenterCrop(224),  # Crop the image to 224x224
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Function to extract features
def extract_features(image_path):
    image = Image.open(image_path)  # Open image
    image = transform(image).unsqueeze(0)  # Apply transformations and add batch dimension

    with torch.no_grad():  # Disable gradient computation
        features = model(image)  # Get the features (model output)
    
    return features.numpy().flatten()  # Flatten the features to a 1D array

# Example: Extract features from an image
image_features_df = pd.DataFrame(columns=["image", "features", "label"])
for i,folder in enumerate(source_folders):
    for image in os.listdir(folder):
        # The first folder contains deforestation images
        if i == 0:
            label = "deforestation"
        else:
            label = "not_deforestation"
        features = extract_features(folder + '/' + image)
        
        data = {"image": image, "features": features, "label": label}
        image_features_df.loc[len(image_features_df.index)] = data 
        
image_features_df.shape
        
    


/Users/maanders1/miniconda3/envs/junction/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/maanders1/miniconda3/envs/junction/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(4, 3)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import pickle


In [23]:
X_test = list(image_features_df["features"].values)
y_test = list(image_features_df["label"].values)

print(f"Testing dataset: {len(X_test)} samples with {len(X_test[0])} features.")

Testing dataset: 4 samples with 1000 features.


In [25]:

with open('classifier.pkl', 'rb') as f:
    svm = pickle.load(f)

# Step 4: Evaluate the model on the test set
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label="deforestation")
precision = precision_score(y_test, y_pred, pos_label="deforestation")
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')


Accuracy: 75.00%
Recall: 100.00%
Precision: 50.00%
